In [ ]:
!pip install -q ultralytics
!pip install -q torchvision torch
!pip install -q pycocotools
!pip install -q matplotlib opencv-python tqdm

from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.2 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
import os
from pathlib import Path
import shutil
import cv2
import yaml
import torch
import torch.nn as nn
from tqdm import tqdm
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
BASE_DIR = Path("/content/drive/MyDrive/VisDroneDataset")

# INPUT (tetap original)
IMAGES_TRAIN = BASE_DIR / "VisDrone2019-DET-train" / "images"
LABELS_TRAIN = BASE_DIR / "VisDrone2019-DET-train" / "annotations"
IMAGES_VAL   = BASE_DIR / "VisDrone2019-DET-val"   / "images"
LABELS_VAL   = BASE_DIR / "VisDrone2019-DET-val"   / "annotations"

# OUTPUT structured 5-class dataset
OUT_ROOT = BASE_DIR / "VisDrone2019_5cls"
OUT_IMG_TRAIN = OUT_ROOT / "images" / "train"
OUT_LBL_TRAIN = OUT_ROOT / "labels" / "train"
OUT_IMG_VAL   = OUT_ROOT / "images" / "val"
OUT_LBL_VAL   = OUT_ROOT / "labels" / "val"

for p in [OUT_IMG_TRAIN, OUT_LBL_TRAIN, OUT_IMG_VAL, OUT_LBL_VAL]:
    p.mkdir(parents=True, exist_ok=True)

SKIP_IMAGES_WITHOUT_TARGET = True

In [ ]:
COMBINED_CLASSES = ["bicycle", "car", "truck", "bus", "motorcycle"]

VIS_CAT_TO_COMBINED = {
    3: 0,   # bicycle
    4: 1,   # car
    5: 1,   # van -> car
    6: 2,   # truck
    9: 3,   # bus
    10: 4,  # motor -> motorcycle
}

def yolo_box_from_xywh(x, y, w, h, W, H):
    xc = x + w/2
    yc = y + h/2
    return xc/W, yc/H, w/W, h/H

In [ ]:
def convert_split(img_dir: Path, ann_dir: Path, out_img_dir: Path, out_lbl_dir: Path):
    img_count = 0
    box_count = [0] * len(COMBINED_CLASSES)

    for img_name in tqdm(sorted(os.listdir(img_dir)), desc=f"Convert {img_dir.name}"):

        if not img_name.lower().endswith(".jpg"):
            continue

        src_img = img_dir / img_name
        ann_path = ann_dir / img_name.replace(".jpg", ".txt")
        if not ann_path.exists():
            continue

        img = cv2.imread(str(src_img))
        if img is None:
            continue
        H, W = img.shape[:2]

        with ann_path.open("r") as f:
            raw = f.readlines()

        yolo_lines = []

        for line in raw:
            parts = [p.strip() for p in line.split(",") if p.strip() != ""]
            if len(parts) < 8:
                continue

            try:
                x, y, w, h, score, cid, trunc, occ = map(float, parts[:8])
            except:
                continue

            cid = int(cid)
            if cid not in VIS_CAT_TO_COMBINED:
                continue

            cls = VIS_CAT_TO_COMBINED[cid]
            xc, yc, wn, hn = yolo_box_from_xywh(x, y, w, h, W, H)

            if wn <= 0 or hn <= 0:
                continue

            yolo_lines.append(f"{cls} {xc:.6f} {yc:.6f} {wn:.6f} {hn:.6f}")
            box_count[cls] += 1

        if SKIP_IMAGES_WITHOUT_TARGET and len(yolo_lines) == 0:
            continue

        dst_img = out_img_dir / img_name
        dst_lbl = out_lbl_dir / img_name.replace(".jpg", ".txt")

        shutil.copy2(src_img, dst_img)
        dst_lbl.write_text("\n".join(yolo_lines))

        img_count += 1

    print(f"\n➡ {img_dir.name}: {img_count} images")
    print("BBox per class:")
    for i, c in enumerate(COMBINED_CLASSES):
        print(f"  {i} ({c}): {box_count[i]}")

    return box_count

# RUN CONVERSION
box_train = convert_split(IMAGES_TRAIN, LABELS_TRAIN, OUT_IMG_TRAIN, OUT_LBL_TRAIN)
box_val   = convert_split(IMAGES_VAL,   LABELS_VAL,   OUT_IMG_VAL,   OUT_LBL_VAL)

Convert images: 100%|██████████| 6471/6471 [3:26:46<00:00,  1.92s/it]



➡ images: 6318 images
BBox per class:
  0 (bicycle): 10480
  1 (car): 169822
  2 (truck): 12875
  3 (bus): 5926
  4 (motorcycle): 29647


Convert images: 100%|██████████| 548/548 [07:55<00:00,  1.15it/s]


➡ images: 546 images
BBox per class:
  0 (bicycle): 1287
  1 (car): 16039
  2 (truck): 750
  3 (bus): 251
  4 (motorcycle): 4886


In [ ]:
yaml_5cls_path = BASE_DIR / "VisDrone2019_5cls.yaml"

yaml_5cls_text = f"""path: {OUT_ROOT.as_posix()}

train: images/train
val: images/val

names:
  0: bicycle
  1: car
  2: truck
  3: bus
  4: motorcycle
"""

yaml_5cls_path.write_text(yaml_5cls_text)
print("\n✓ YAML 5-class saved:", yaml_5cls_path)
print(yaml_5cls_text)


✓ YAML 5-class saved: /content/drive/MyDrive/VisDroneDataset/VisDrone2019_5cls.yaml
path: /content/drive/MyDrive/VisDroneDataset/VisDrone2019_5cls

train: images/train
val: images/val

names:
  0: bicycle
  1: car
  2: truck
  3: bus
  4: motorcycle



In [ ]:
FOCUS_ROOT = BASE_DIR / "VisDrone2019_5cls_focus_2roda"
FOCUS_IMG_TRAIN = FOCUS_ROOT / "images" / "train_2roda"
FOCUS_LBL_TRAIN = FOCUS_ROOT / "labels" / "train_2roda"
FOCUS_IMG_VAL   = FOCUS_ROOT / "images" / "val"
FOCUS_LBL_VAL   = FOCUS_ROOT / "labels" / "val"

for p in [FOCUS_IMG_TRAIN, FOCUS_LBL_TRAIN, FOCUS_IMG_VAL, FOCUS_LBL_VAL]:
    p.mkdir(parents=True, exist_ok=True)

files_copied = 0
bicycle_boxes = 0
moto_boxes = 0

for lbl_path in OUT_LBL_TRAIN.glob("*.txt"):
    lines = lbl_path.read_text().splitlines()

    if not any(l.startswith("0 ") or l.startswith("4 ") for l in lines):
        continue

    img_name = lbl_path.stem + ".jpg"
    src_img = OUT_IMG_TRAIN / img_name
    if not src_img.exists():
        continue

    shutil.copy2(src_img, FOCUS_IMG_TRAIN / img_name)
    shutil.copy2(lbl_path, FOCUS_LBL_TRAIN / lbl_path.name)

    bicycle_boxes += sum(1 for l in lines if l.startswith("0 "))
    moto_boxes    += sum(1 for l in lines if l.startswith("4 "))

    files_copied += 1

# SALIN VAL DARI DATASET 5-CLASS
shutil.copytree(OUT_IMG_VAL, FOCUS_IMG_VAL, dirs_exist_ok=True)
shutil.copytree(OUT_LBL_VAL, FOCUS_LBL_VAL, dirs_exist_ok=True)

print("\nSubset fokus 2 roda:")
print(" images:", files_copied)
print(" bicycle boxes:", bicycle_boxes)
print(" motorcycle boxes:", moto_boxes)



Subset fokus 2 roda:
 images: 4573
 bicycle boxes: 10480
 motorcycle boxes: 29647


In [ ]:
yaml_focus_path = BASE_DIR / "VisDrone2019_5cls_focus_2roda.yaml"

yaml_focus_text = f"""path: {FOCUS_ROOT.as_posix()}

train: images/train_2roda
val: images/val

names:
  0: bicycle
  1: car
  2: truck
  3: bus
  4: motorcycle
"""

yaml_focus_path.write_text(yaml_focus_text)
print("\n✓ YAML Fokus 2 Roda saved:", yaml_focus_path)
print(yaml_focus_text)


✓ YAML Fokus 2 Roda saved: /content/drive/MyDrive/VisDroneDataset/VisDrone2019_5cls_focus_2roda.yaml
path: /content/drive/MyDrive/VisDroneDataset/VisDrone2019_5cls_focus_2roda

train: images/train_2roda
val: images/val

names:
  0: bicycle
  1: car
  2: truck
  3: bus
  4: motorcycle



In [ ]:
import requests

url = "https://huggingface.co/Ultralytics/YOLO11/resolve/main/yolo11n.pt"
r = requests.get(url)
open("yolov11n.pt", "wb").write(r.content)
print("\n✓ Downloaded yolov11n.pt")


✓ Downloaded yolov11n.pt


In [ ]:
class CBAM(nn.Module):
    def __init__(self, ch, reduction=16, k=7):
        super().__init__()
        self.ca = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(ch, ch//reduction, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(ch//reduction, ch, 1, bias=False),
            nn.Sigmoid()
        )
        self.sa = nn.Sequential(
            nn.Conv2d(2, 1, k, padding=k//2, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x * self.ca(x)
        avg = torch.mean(x, dim=1, keepdim=True)
        mx, _ = torch.max(x, dim=1, keepdim=True)
        x = x * self.sa(torch.cat([avg, mx], dim=1))
        return x

model = YOLO("yolov11n.pt")

cbam_layers = [2, 5, 8]
for idx in cbam_layers:
    L = model.model.model[idx]
    out_ch = L.cv1.conv.out_channels if hasattr(L,"cv1") else L.conv.out_channels
    model.model.model[idx] = nn.Sequential(L, CBAM(out_ch))


In [ ]:
yaml_5cls_path = BASE_DIR / "VisDrone2019_5cls.yaml"

results_base = model.train(
    data=str(yaml_5cls_path),
    epochs=80,
    imgsz=1024,
    batch=6,
    optimizer='SGD',
    lr0=0.005,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=5,
    val=True,
    device="cuda",
    project=str(BASE_DIR),
    name="yolov11_visdrone_5cls_cbam",
    exist_ok=True,
)


Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/VisDroneDataset/VisDrone2019_5cls.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11_visdrone_5cls_cbam, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, over

In [ ]:
yaml_focus_path = BASE_DIR / "VisDrone2019_5cls_focus_2roda.yaml"
best_base = BASE_DIR / "yolov11_visdrone_5cls_cbam" / "weights" / "best.pt"
model_focus = YOLO(str(best_base))

results_focus = model_focus.train(
    data=str(yaml_focus_path),
    epochs=50,
    imgsz=1024,
    batch=4,
    lr0=0.001,
    optimizer='SGD',
    warmup_epochs=3,
    val=True,
    device="cuda",
    project=str(BASE_DIR),
    name="yolov11_visdrone_5cls_bikemoto_ft",
    exist_ok=True,
)

Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/VisDroneDataset/VisDrone2019_5cls_focus_2roda.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/VisDroneDataset/yolov11_visdrone_5cls_cbam/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11_visd

In [ ]:
trained_dir = BASE_DIR / "trained_models"
trained_dir.mkdir(exist_ok=True)

src_best = BASE_DIR / "yolov11_visdrone_5cls_bikemoto_ft" / "weights" / "best.pt"
dst_best = trained_dir / "yolov11n_visdrone_5cls_bikemoto_ft.pt"

shutil.copy2(src_best, dst_best)

print("\n===================================================")
print("✓ FINAL MODEL SAVED TO:")
print(dst_best)
print("===================================================")


✓ FINAL MODEL SAVED TO:
/content/drive/MyDrive/VisDroneDataset/trained_models/yolov11n_visdrone_5cls_bikemoto_ft.pt
